In [1]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -U pysal
!pip install geopandas
!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric-temporal
!pip install --upgrade --force-reinstall git+https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git to /tmp/pip-req-build-uc0pmi8f
  Running command git clone --filter=blob:none --quiet https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git /tmp/pip-req-build-uc0pmi8f
  Resolved https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git to commit 76ede78745fb93811bb87520d9fcd457a0a02934
  Preparing metadata (setup.py) ... 

In [ ]:
print("Ok")

In [ ]:
from google.colab import runtime
import torch
import gc
from dataset.covidBR_dataset import *
from model.covid_model import *
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric.loader import GraphSAINTRandomWalkSampler, NeighborLoader
from tqdm import tqdm

In [ ]:
def free_cache():
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
loader = CovidDatasetLoader(method="other")
dataset = loader.get_dataset(lags=4)
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
dataset[0].x.shape[1]

In [ ]:
filter_size = 3
train_model = True
num_feats = dataset[0].x.shape[1]
gamma=1

In [ ]:
device = "cpu"
dtype = torch.FloatTensor
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    device = "cuda"
if device == "cpu" and train_model:
    import torch_directml
    dml = torch_directml.device()
device

In [ ]:
if train_model:
    model = RecurrentGCN(num_features = num_feats, num_filters = filter_size).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    model.train()

    for epoch in tqdm(range(10)):
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat,A_h,A_z,A_r  = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            cost = torch.mean((y_hat-snapshot.y)**2)\
            + gamma*(torch.norm(A_h,p=1) + torch.norm(A_z,p=1) + torch.norm(A_r,p=1))
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        if epoch % 10 == 0:
            torch.save(model.state_dict(), "./model_weights_ADCRNN")
            # torch.save(model, "./the_whole_model")
    torch.save(model, "./the_whole_model_ADCRNN")
    free_cache()
else:
    model = get_model(False)
    model.to(device)

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
def check_mem():
  #Additional Info when using cuda
  if device == 'cuda':
      print(torch.cuda.get_device_name(0))
      print('Memory Usage:')
      print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
      print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
check_mem()

In [ ]:
# free_cache()

In [ ]:
model.eval()
cost = 0
# test_dataset.to(device)
for time, snapshot in enumerate(test_dataset):
    snapshot.to(device)
    y_hat,_,_,_ = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2).item()
    cost = cost / (time+1)
    del snapshot
    free_cache()
    # check_mem()

In [ ]:
print("MSE: {:.4f}".format(cost))

In [ ]:
time

In [ ]:
runtime.unassign()